In [1]:
import pandas as pd
from utils.match_prediction import RAW_PRO_GAMES_FILE


df = pd.read_parquet(RAW_PRO_GAMES_FILE)

df.head()

,golgg_id,champion_ids,gameVersionMajorPatch,gameVersionMinorPatch,gameDuration,blueTeamName,redTeamName,tournamentName,team_100_win,team_100_TOP_totalGold_at_900000,...,team_200_BOTTOM_deaths_at_900000,team_200_BOTTOM_deaths_at_1200000,team_200_BOTTOM_assists_at_900000,team_200_BOTTOM_assists_at_1200000,team_200_UTILITY_kills_at_900000,team_200_UTILITY_kills_at_1200000,team_200_UTILITY_deaths_at_900000,team_200_UTILITY_deaths_at_1200000,team_200_UTILITY_assists_at_900000,team_200_UTILITY_assists_at_1200000
0,65047,"[31, 57, 126, 202, 432, 150, 876, 18, 15, 43]",15,5,1717,Los Ratones,Team Phantasma,EMEA Masters 2025 Winter,1,4990.0,...,2.0,1.0,2.0,2.0,3.0,2.0,6.0,4.0,4.0,3.0
1,65046,"[254, 56, 61, 498, 497, 897, 234, 163, 51, 89]",15,5,2177,Los Ratones,Team Phantasma,EMEA Masters 2025 Winter,1,5175.0,...,3.0,3.0,1.0,1.0,0.0,0.0,1.0,7.0,4.0,4.0
2,65045,"[79, 5, 268, 42, 526, 78, 143, 777, 81, 111]",15,5,2239,Los Ratones,Team Phantasma,EMEA Masters 2025 Winter,1,5949.0,...,1.0,1.0,4.0,7.0,1.0,1.0,4.0,6.0,2.0,6.0
3,65043,"[58, 234, 84, 18, 53, 897, 56, 893, 15, 43]",15,5,1732,Ici Japon Corp. Esport,Karmine Corp Blue,EMEA Masters 2025 Winter,1,5828.0,...,3.0,3.0,1.0,1.0,0.0,0.0,6.0,6.0,1.0,1.0
4,65042,"[799, 5, 103, 42, 111, 78, 76, 777, 498, 89]",15,5,1867,Ici Japon Corp. Esport,Karmine Corp Blue,EMEA Masters 2025 Winter,0,4794.0,...,2.0,2.0,3.0,8.0,0.0,0.0,4.0,5.0,5.0,13.0


In [2]:
for col in df.columns:
    print(col)

golgg_id
champion_ids
gameVersionMajorPatch
gameVersionMinorPatch
gameDuration
blueTeamName
redTeamName
tournamentName
team_100_win
team_100_TOP_totalGold_at_900000
team_100_TOP_totalGold_at_1200000
team_100_TOP_creepScore_at_900000
team_100_TOP_creepScore_at_1200000
team_100_JUNGLE_totalGold_at_900000
team_100_JUNGLE_totalGold_at_1200000
team_100_JUNGLE_creepScore_at_900000
team_100_JUNGLE_creepScore_at_1200000
team_100_MIDDLE_totalGold_at_900000
team_100_MIDDLE_totalGold_at_1200000
team_100_MIDDLE_creepScore_at_900000
team_100_MIDDLE_creepScore_at_1200000
team_100_BOTTOM_totalGold_at_900000
team_100_BOTTOM_totalGold_at_1200000
team_100_BOTTOM_creepScore_at_900000
team_100_BOTTOM_creepScore_at_1200000
team_100_UTILITY_totalGold_at_900000
team_100_UTILITY_totalGold_at_1200000
team_100_UTILITY_creepScore_at_900000
team_100_UTILITY_creepScore_at_1200000
team_200_TOP_totalGold_at_900000
team_200_TOP_totalGold_at_1200000
team_200_TOP_creepScore_at_900000
team_200_TOP_creepScore_at_1200000


In [ ]:
# count nans
df["team_200_BOTTOM_deaths_at_900000"].isna().sum()

In [ ]:
len(df)

In [ ]:
smalled_golgg_id = df["golgg_id"].min()
print(smalled_golgg_id)

In [ ]:
# type of champion_ids
df["champion_ids"].dtype

In [ ]:
# find when champion_ids is not a list of 10 integers
df[
    ~df["champion_ids"].apply(
        lambda x: isinstance(x.tolist(), list) and len(x.tolist()) == 10
    )
]

In [ ]:
len(
    df[
        ~df["champion_ids"].apply(
            lambda x: isinstance(x.tolist(), list) and len(x.tolist()) == 10
        )
    ]
)

In [2]:
# Calculate statistics for each team
team_stats = {}

# Process blue team (team 100) games
for _, row in df.iterrows():
    blue_team = row["blueTeamName"]
    red_team = row["redTeamName"]
    blue_win = row["team_100_win"]

    # Update blue team stats
    if blue_team not in team_stats:
        team_stats[blue_team] = {"wins": 0, "total_games": 0}
    team_stats[blue_team]["wins"] += blue_win
    team_stats[blue_team]["total_games"] += 1

    # Update red team stats
    if red_team not in team_stats:
        team_stats[red_team] = {"wins": 0, "total_games": 0}
    team_stats[red_team]["wins"] += 1 - blue_win  # red team wins when blue team loses
    team_stats[red_team]["total_games"] += 1

# Calculate winrates and store in a list
winrates = []
for team, stats in team_stats.items():
    winrate = (stats["wins"] / stats["total_games"]) * 100
    winrates.append((team, winrate, stats["total_games"]))

# Create buckets
buckets = {f"{i}-{i+10}%": 0 for i in range(0, 100, 10)}

# Sort teams into buckets
for team, winrate, games in winrates:
    bucket_key = f"{int(winrate // 10 * 10)}-{int(winrate // 10 * 10 + 10)}%"
    buckets[bucket_key] += 1

# Print results
print("Winrate distribution (minimum 5 games played):")
for bucket, count in sorted(buckets.items(), reverse=True):
    if count > 0:
        print(f"{bucket}: {count} teams")

print("\nTop 5 teams by winrate (minimum 5 games played):")
filtered_winrates = [x for x in winrates if x[2] >= 5]  # Only teams with 5+ games
filtered_winrates.sort(key=lambda x: x[1], reverse=True)
for team, winrate, games in filtered_winrates[:5]:
    print(f"{team}: {winrate:.1f}% ({games} games)")

Winrate distribution (minimum 5 games played):
90-100%: 3 teams
80-90%: 5 teams
70-80%: 24 teams
60-70%: 57 teams
50-60%: 128 teams
40-50%: 118 teams
30-40%: 104 teams
20-30%: 50 teams
10-20%: 22 teams
0-10%: 21 teams

Top 5 teams by winrate (minimum 5 games played):
QT DIG: 90.9% (11 games)
REJECT: 90.9% (11 games)
Partizan Sangal: 90.9% (11 games)
Los Ratones: 86.8% (38 games)
DarkZero Dragonsteel: 86.7% (15 games)
